In [1]:
import tensorflow as tf
import cv2
import numpy as np
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import gen_array_ops
from tensorflow.python.framework import ops
import math

In [2]:
def maxout(inputs, num_units, axis):
    inputs = ops.convert_to_tensor(inputs)
    shape = inputs.get_shape().as_list()
    num_channels = shape[axis]
    shape[axis] = -1
    shape += [num_channels // num_units]
    for i in range(len(shape)):
        if shape[i] is None:
                shape[i] = gen_array_ops.shape(inputs)[i]
    outputs = math_ops.reduce_max(gen_array_ops.reshape(inputs, shape), -1, keep_dims=False)
    return outputs

In [3]:
img = cv2.resize(cv2.cvtColor(cv2.imread("img/canon.jpg"), cv2.COLOR_BGR2RGB), (16, 16))
img_tensor = tf.reshape(tf.convert_to_tensor(img, dtype=tf.float32), [1, 16, 16, 3])
filters_1 = tf.reshape(tf.constant(np.load("numpy_parameters/conv1_0.npy"), name="filters_1"), [5, 5, 3, 20])
bias_1 = tf.constant(np.load("numpy_parameters/conv1_1.npy"), name="bias_1")
conv_1 = tf.nn.conv2d(img_tensor, filters_1, [1, 1, 1, 1], padding="VALID")
output_1 =  tf.nn.relu(tf.nn.bias_add(conv_1, bias_1))
reshape_1 = tf.reshape(output_1, [1, 12, 12, 20])
max_1 = maxout(reshape_1, 4, 3)
reshape_2 = tf.reshape(max_1, [1, 12, 12, 4])
filters_2 = tf.reshape(tf.constant(np.load("numpy_parameters/conv2-1x1_0.npy"), name="filters_2"), [1, 1, 4, 16])
bias_2 = tf.constant(np.load("numpy_parameters/conv2-1x1_1.npy"), name="bias_2")
conv_2 = tf.nn.conv2d(reshape_2, filters_2, [1, 1, 1, 1], padding="VALID")
output_2 =  tf.nn.relu(tf.nn.bias_add(conv_2, bias_2))
filters_3 = tf.reshape(tf.constant(np.load("numpy_parameters/conv2-3x3_0.npy"), name="filters_3"), [3, 3, 4, 16])
bias_3 = tf.constant(np.load("numpy_parameters/conv2-3x3_1.npy"), name="bias_3")
conv_3 = tf.nn.conv2d(reshape_2, filters_3, [1, 1, 1, 1], padding="SAME")
output_3 =  tf.nn.relu(tf.nn.bias_add(conv_3, bias_3))
filters_4 = tf.reshape(tf.constant(np.load("numpy_parameters/conv2-5x5_0.npy"), name="filters_4"), [5, 5, 4, 16])
bias_4 = tf.constant(np.load("numpy_parameters/conv2-5x5_1.npy"), name="bias_4")
conv_4 = tf.nn.conv2d(reshape_2, filters_4, [1, 1, 1, 1], padding="SAME")
output_4 =  tf.nn.relu(tf.nn.bias_add(conv_4, bias_4))
filters_5 = tf.reshape(tf.constant(np.load("numpy_parameters/conv2-7x7_0.npy"), name="filters_5"), [7, 7, 4, 16])
bias_5 = tf.constant(np.load("numpy_parameters/conv2-7x7_1.npy"), name="bias_5")
conv_5 = tf.nn.conv2d(reshape_2, filters_5, [1, 1, 1, 1], padding="SAME")
output_5 =  tf.nn.relu(tf.nn.bias_add(conv_5, bias_5))
output_6 = tf.concat([output_2, output_3, output_4, output_5], axis=3)
output_7 = tf.nn.relu(output_6)
output_8  = tf.nn.max_pool(output_7, [1, 8, 8, 1], [1, 1, 1, 1], padding='VALID')
filters_6 = np.load("numpy_parameters/ip1_0.npy").reshape((5, 5, 64, 1))
output_9 = tf.nn.conv2d(output_8, filters_6, [1, 1, 1, 1], padding="VALID")
#transmission = None
with tf.Session() as sess:
    output_9 = sess.run(output_9)
    print(output_9.shape)
    #output_8, transmission = sess.run([output_8, output_9])
    #print(output_8.shape)
    #transmission = transmission.reshape(525, 600)

(1, 1, 1, 1)


In [3]:
def TransmissionEstimate(im_path, height, width): 
    img = cv2.resize(cv2.cvtColor(cv2.imread("img/canon.jpg"), cv2.COLOR_BGR2RGB), (320,320))
    print(img.shape)
    img  = (img * 1.0) / (255 * 1.0)
    npad = ((7,8), (7,8), (0,0))
    img = np.pad(img, npad, 'symmetric')
    print(img.shape)
    img_tensor = tf.reshape(tf.convert_to_tensor(img, dtype=tf.float32), [1, height+15, width+15, 3])
    print(img_tensor)
    filters_1 = tf.reshape(tf.constant(np.load("numpy_parameters/conv1_0.npy"), name="filters_1"), [5, 5, 3, 20])
    bias_1 = tf.constant(np.load("numpy_parameters/conv1_1.npy"), name="bias_1")
    conv_1 = tf.nn.conv2d(img_tensor, filters_1, [1, 1, 1, 1], padding="VALID")
    output_1 =  tf.nn.relu(tf.nn.bias_add(conv_1, bias_1))
    
    print(output_1)
    
    reshape_1 = tf.reshape(output_1, [1, height+11, width+11, 20])
    max_1 = maxout(reshape_1, 4, 3)
    reshape_2 = tf.reshape(max_1, [1, height+11, width+11, 4])
    print(reshape_2)
    
    filters_2 = tf.reshape(tf.constant(np.load("numpy_parameters/conv2-1x1_0.npy"), name="filters_2"), [1, 1, 4, 16])
    bias_2 = tf.constant(np.load("numpy_parameters/conv2-1x1_1.npy"), name="bias_2")
    conv_2 = tf.nn.conv2d(reshape_2, filters_2, [1, 1, 1, 1], padding="VALID")
    output_2 =  tf.nn.relu(tf.nn.bias_add(conv_2, bias_2))
    print(output_2)
    
    filters_3 = tf.reshape(tf.constant(np.load("numpy_parameters/conv2-3x3_0.npy"), name="filters_3"), [3, 3, 4, 16])
    bias_3 = tf.constant(np.load("numpy_parameters/conv2-3x3_1.npy"), name="bias_3")
    conv_3 = tf.nn.conv2d(reshape_2, filters_3, [1, 1, 1, 1], padding="SAME")
    output_3 =  tf.nn.relu(tf.nn.bias_add(conv_3, bias_3))
    
    filters_4 = tf.reshape(tf.constant(np.load("numpy_parameters/conv2-5x5_0.npy"), name="filters_4"), [5, 5, 4, 16])
    bias_4 = tf.constant(np.load("numpy_parameters/conv2-5x5_1.npy"), name="bias_4")
    conv_4 = tf.nn.conv2d(reshape_2, filters_4, [1, 1, 1, 1], padding="SAME")
    output_4 =  tf.nn.relu(tf.nn.bias_add(conv_4, bias_4))
    
    filters_5 = tf.reshape(tf.constant(np.load("numpy_parameters/conv2-7x7_0.npy"), name="filters_5"), [7, 7, 4, 16])
    bias_5 = tf.constant(np.load("numpy_parameters/conv2-7x7_1.npy"), name="bias_5")
    conv_5 = tf.nn.conv2d(reshape_2, filters_5, [1, 1, 1, 1], padding="SAME")
    output_5 =  tf.nn.relu(tf.nn.bias_add(conv_5, bias_5))
    
    output_6 = tf.concat([output_4, output_3, output_4, output_5], axis=3)
    output_7 = tf.nn.relu(output_6)
    print(output_6)
    output_8  = tf.nn.max_pool(output_7, [1, 8, 8, 1], [1, 1, 1, 1], padding='VALID')
    
    print(output_8)
    
    final_conv_1 = tf.layers.conv2d(inputs=output_8,
                                    filters=1,
                                    strides=(4, 4),
                                    kernel_size=[8, 8],
                                    padding="VALID",
                                    activation=tf.nn.relu,
                                    name="DehazeNet/final_conv_1")
    
    #filters_6 = tf.reshape(tf.constant(np.load("numpy_parameters/ip1_0.npy"), name="filters_6"), [5, 5, 64, 1])
    #bias_6 = tf.constant(np.load("numpy_parameters/ip1_1.npy"), name="bias_6")
    #conv_6 = tf.nn.conv2d(output_8, filters_6, [1, 2, 2, 1], padding="VALID")
    
    #final_output = tf.nn.relu(tf.nn.bias_add(conv_6, bias_6))
    
    final_conv_2 = tf.layers.conv2d(inputs=tf.nn.max_pool(final_conv_1, [1, 8, 8, 1], [1, 8, 8, 1], padding='VALID'),
                                    filters=1,
                                    strides=(4, 4),
                                    kernel_size=[8, 8],
                                    padding="VALID",
                                    activation=tf.nn.relu,
                                    name="DehazeNet/final_conv_2")
    
    #final_conv_3 = tf.layers.conv2d(inputs=tf.nn.max_pool(final_conv, [1, 8, 8, 1], [1, 2, 2, 1], padding='VALID'),
    #                                filters=1,
    #                                strides=(2, 2),
    #                                kernel_size=[4, 4],
    #                                padding="VALID",
    #                                activation=tf.nn.relu,
    #                                name="DehazeNet/final_conv_2")
    transmission = None
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        sess.run(tf.tables_initializer())
        final_output_numpy = sess.run(final_conv_2)
        print(final_output_numpy.shape)
    return transmission

In [4]:
def DarkChannel(im,sz):
    b,g,r = cv2.split(im)
    dc = cv2.min(cv2.min(r,g),b)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(sz,sz))
    dark = cv2.erode(dc,kernel)
    return dark

In [5]:
def AtmLight(im,dark):
    [h,w] = im.shape[:2]
    imsz = h*w
    numpx = int(max(math.floor(imsz/1000),1))
    darkvec = dark.reshape(imsz,1)
    imvec = im.reshape(imsz,3)
    indices = darkvec.argsort()
    indices = indices[imsz-numpx::]
    atmsum = np.zeros([1,3])
    for ind in range(1,numpx):
        atmsum = atmsum + imvec[indices[ind]]
    A = atmsum / numpx
    return A

def Guidedfilter(im,p,r,eps):
    mean_I = cv2.boxFilter(im,cv2.CV_64F,(r,r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F,(r,r))
    mean_Ip = cv2.boxFilter(im*p,cv2.CV_64F,(r,r))
    cov_Ip = mean_Ip - mean_I*mean_p
    mean_II = cv2.boxFilter(im*im,cv2.CV_64F,(r,r))
    var_I   = mean_II - mean_I*mean_I
    a = cov_Ip/(var_I + eps)
    b = mean_p - a*mean_I
    mean_a = cv2.boxFilter(a,cv2.CV_64F,(r,r))
    mean_b = cv2.boxFilter(b,cv2.CV_64F,(r,r))
    q = mean_a*im + mean_b
    return q

def TransmissionRefine(im,et):
    gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray)/255
    r = 60
    eps = 0.0001
    t = Guidedfilter(gray,et,r,eps)
    return t

def Recover(im,t,A,tx = 0.1):
    res = np.empty(im.shape,im.dtype)
    t = cv2.max(t,tx)
    for ind in range(0,3):
        res[:,:,ind] = (im[:,:,ind]-A[0,ind])/t + A[0,ind]
    return res

In [6]:
im_path = "img/canon.jpg"
#src = cv2.resize(cv2.imread(im_path))

#height = src.shape[0]
#width = src.shape[1]
#print(height)
#print(width)
#I = src/255.0
#dark = DarkChannel(I,15)
#A = AtmLight(I,dark)
te = TransmissionEstimate(im_path, 320, 320)
#t = TransmissionRefine(src,te)
#J = Recover(I,t,A,0.1)
#cv2.imshow('TransmissionEstimate',te)
#cv2.imshow('Origin',src)
#cv2.imshow('Dehaze',J)
#cv2.waitKey(0)

(320, 320, 3)
(335, 335, 3)
Tensor("Reshape:0", shape=(1, 335, 335, 3), dtype=float32)
Tensor("Relu:0", shape=(1, 331, 331, 20), dtype=float32)
Tensor("Reshape_4:0", shape=(1, 331, 331, 4), dtype=float32)
Tensor("Relu_1:0", shape=(1, 331, 331, 16), dtype=float32)
Tensor("concat:0", shape=(1, 331, 331, 64), dtype=float32)
Tensor("MaxPool:0", shape=(1, 324, 324, 64), dtype=float32)
(1, 1, 1, 1)
